> # **[2020 학습활동]**


---



> # **Day 1. 사전 훈련된 컨브넷(VGG16) 활용**
2020.05.26. Fri

# **ABSTRACT**: ImageNet data로 사전 훈련된 ConvNet 모델(VGG16)을 활용하여 이미지 이진 분류 모델(고양이 vs 개) 만들어보기
# **KEYWORD**: ConvNet, VGG16, ImageNet, Feature Exctraction, Classification Layer



> # **1. Data 다운로드 및 전처리**
출처: kaggle 강아지 vs 고양이 데이터 (https://www.kaggle.com/c/dogs-vs-cats/data)

In [0]:
# 데이터가 이미 있을경우 삭제
!rm -rf datasets
# 데이터 다운로드
!git clone http://gitlab.com/jooncco/datasets
# 압축 해제
!cd datasets && unzip dogs-vs-cats.zip && mkdir cats_and_dogs && mv ./train.zip ./cats_and_dogs/train.zip && cd cats_and_dogs && unzip train.zip && rm train.zip

In [0]:
!rm -rf ./datasets/cats_and_dogs_small

import os, shutil

# 이미지 파일이 있는 colab 가상 파일시스템의 경로
original_dataset_dir= './datasets/cats_and_dogs/train'

base_dir= './datasets/cats_and_dogs_small'            # 소규모 데이터셋을 저장할 경로
os.mkdir(base_dir)

# 훈련/검증/테스트 데이터 디렉토리 생성
train_dir= os.path.join(base_dir, 'train')            # 훈련 데이터 directory
os.mkdir(train_dir)
validation_dir= os.path.join(base_dir, 'validation')  # 검증 데이터 directory
os.mkdir(validation_dir)
test_dir= os.path.join(base_dir, 'test')              # 테스트 데이터 directory
os.mkdir(test_dir)

train_cats_dir= os.path.join(train_dir, 'cats')                 # 훈련용 고양이 사진 directory
os.mkdir(train_cats_dir)
train_dogs_dir= os.path.join(train_dir, 'dogs')                 # 훈련용 강아지 사진 directory
os.mkdir(train_dogs_dir)
validation_cats_dir= os.path.join(validation_dir, 'cats')       # 검증용 고양이 사진 directory
os.mkdir(validation_cats_dir)
validation_dogs_dir= os.path.join(validation_dir, 'dogs')       # 검증용 강아지 사진 directory
os.mkdir(validation_dogs_dir)
test_cats_dir= os.path.join(test_dir, 'cats')                   # 테스트용 고양이 사진 directory
os.mkdir(test_cats_dir)
test_dogs_dir= os.path.join(test_dir, 'dogs')                   # 테스트용 강아지 사진 directory
os.mkdir(test_dogs_dir)


# 고양이 사진들을 훈련/검증/테스트 용으로 분할하기
fnames= ['cat.{}.jpg'.format(i) for i in range(1000)]                # 처음 1000장의 고양이 사진을 train_cats_dir에 복사
for fname in fnames:
  src= os.path.join(original_dataset_dir,fname)     # 원본 경로
  dst= os.path.join(train_cats_dir,fname)           # 옮겨질 경로
  shutil.copyfile(src,dst)

fnames= ['cat.{}.jpg'.format(i) for i in range(1000,1500)]           # 다음 500장의 고양이 사진을 validation_cats_dir에 복사
for fname in fnames:
  src= os.path.join(original_dataset_dir,fname)     # 원본 경로
  dst= os.path.join(validation_cats_dir,fname)      # 옮겨질 경로
  shutil.copyfile(src,dst)

fnames= ['cat.{}.jpg'.format(i) for i in range(1500,2000)]           # 다음 500장의 고양이 사진을 test_cats_dir 복사
for fname in fnames:
  src= os.path.join(original_dataset_dir,fname)     # 원본 경로
  dst= os.path.join(test_cats_dir,fname)            # 옮겨질 경로
  shutil.copyfile(src,dst)

# 강아지 사진들을 훈련/검증/테스트 용으로 분할하기
fnames= ['dog.{}.jpg'.format(i) for i in range(1000)]                # 처음 1000장의 강아지 사진을 train_cats_dir에 복사
for fname in fnames:
  src= os.path.join(original_dataset_dir,fname)     # 원본 경로
  dst= os.path.join(train_dogs_dir,fname)           # 옮겨질 경로
  shutil.copyfile(src,dst)

fnames= ['dog.{}.jpg'.format(i) for i in range(1000,1500)]           # 다음 500장의 강아지 사진을 validation_cats_dir에 복사
for fname in fnames:
  src= os.path.join(original_dataset_dir,fname)     # 원본 경로
  dst= os.path.join(validation_dogs_dir,fname)      # 옮겨질 경로
  shutil.copyfile(src,dst)

fnames= ['dog.{}.jpg'.format(i) for i in range(1500,2000)]           # 다음 500장의 강아지 사진을 test_cats_dir 복사
for fname in fnames:
  src= os.path.join(original_dataset_dir,fname)     # 원본 경로
  dst= os.path.join(test_dogs_dir,fname)            # 옮겨질 경로
  shutil.copyfile(src,dst)

# 잘 옮겨졌나?
print("훈련용 고양이 이미지 전체 개수: ", len(os.listdir(train_cats_dir)))
print("검증용 고양이 이미지 전체 개수: ", len(os.listdir(validation_cats_dir)))
print("테스트용 고양이 이미지 전체 개수: ", len(os.listdir(test_cats_dir)))
print("훈련용 강아지 이미지 전체 개수: ", len(os.listdir(train_dogs_dir)))
print("검증용 강아지 이미지 전체 개수: ", len(os.listdir(validation_dogs_dir)))
print("테스트용 고양이 이미지 전체 개수: ", len(os.listdir(test_dogs_dir)))

> # **2. ConvNet(VGG16) 준비**

In [0]:
from keras.applications import VGG16

conv_base= VGG16(weights= 'imagenet',          # imagenet 데이터로 미리 학습된 weight들을 로드
                 include_top= False,           # Fully connected classifier 층은 제외
                 input_shape= (150,150,3))     # 150 x 150 pixel의 rgb값이 들어있는 텐서를 input으로 받음
conv_base.summary()       # 내 모델을 보여줘

> # **3. 특성 추출 (Feature Extraction)**

In [0]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir= './datasets/cats_and_dogs_small'              # 루트 디렉토리
train_dir= os.path.join(base_dir, 'train')              # 훈련 데이터 디렉토리
validation_dir= os.path.join(base_dir, 'validation')    # 훈련 검증 데이터 디렉토리
test_dir= os.path.join(base_dir, 'test')                # 테스트 데이터 디렉토리

datagen= ImageDataGenerator(rescale=1.0/255)            # 픽셀 하나를 2^8 (1byte)의 범위를 갖는 rgb 값으로 환산하겠다.
batch_size= 20                                          # 샘플 데이터를 이 크기의 배치로 분할


# 이미지 데이터에서 추출한 feature 들을 리턴해주는 함수
def extract_features(directory, sample_count):
  features= np.zeros(shape= (sample_count, 4, 4, 512))            # 데이터들을 모델이 처리한 결과(features)가 담길 numpy array
  labels= np.zeros(shape= (sample_count))                         # label 이 담길 numpy array
  generator= datagen.flow_from_directory(directory,
                                         target_size= (150,150),  # input data: 150 x 150
                                         batch_size= batch_size,
                                         class_mode= 'binary')    # 이중 분류 (강아지 vs 고양이)

  i= 0
  for inputs_batch, labels_batch in generator:
    features_batch= conv_base.predict(inputs_batch)                 # 배치 하나의 예측 결과를 담은 numpy array
    features[i * batch_size : (i+1) * batch_size]= features_batch   # write
    labels[i * batch_size : (i+1) * batch_size]= labels_batch       # write
    
    i += 1
    if (i * batch_size >= sample_count):
      break                                 # 무한루프 탈출
  return features, labels

train_features, train_labels= extract_features(train_dir, 2000)                   # 훈련을 위한 feature extraction 결과
validation_features, validation_labels= extract_features(validation_dir, 1000)    # 훈련 검증을 위한 feature extraction 결과
test_features, test_labels= extract_features(test_dir, 1000)                      # 테스트를 위한 feature extraction 결과



> # **4. 분류기 정의 (Classification Output Layer)**

In [0]:
from keras import models
from keras import layers            # layer들을 레고 블럭처럼
from keras import optimizers        # weight 업데이트 해주는 녀석


model= models.Sequential()
model.add(layers.Dense(256, activation= 'relu', input_dim= 4*4*512))            # layer 1: 밀집       ##################### 1.수정가능 (activation)
model.add(layers.Dropout(0.5))                                                  # layer 2: Dropout    ##################### 2.수정가능 (Dropout argument)
model.add(layers.Dense(1, activation='sigmoid'))                                # layer 3: 밀집       ##################### 3.수정가능 (activation)
model.compile(optimizer= optimizers.RMSprop(lr= 2e-5),                          # optimizer 정의      ##################### 4.수정가능 (lr) <= 현재값: 2 x 10^(-5)
              loss= 'binary_crossentropy',                                      # loss function 정의  ##################### 5.수정가능 (loss)
              metrics= ['acc'])
model.summary()                 # 내 모델을 보여줘



> # **5. 훈련**

In [0]:

# 분류기의 input_dim이 (4 x 4 x 512) 이기 때문에 feature를 (sample_count x 8192)로 펼쳐준다.
train_features= np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features= np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features= np.reshape(test_features, (1000, 4 * 4 * 512))

# 학습
learn_history= model.fit(train_features, train_labels,
                         epochs= 30,                                                            # 학습 반복 회수(epoch) 설정      ##################### 6.수정가능 (epochs)
                         batch_size= 20,                                                        # batch 사이즈 설정               ##################### 7.수정가능 (batch_size)
                         validation_data= (validation_features, validation_labels))



> # **6. 훈련 결과 Plot**

In [0]:
import matplotlib.pyplot as plt

acc= learn_history.history['acc']
val_acc= learn_history.history['val_acc']
loss= learn_history.history['loss']
val_loss= learn_history.history['val_loss']

# 그래프의 x축
epochs= range(1, len(acc)+1)

# epoch별 정확도 변화 그래프
plt.plot(epochs, acc, 'bo', label= 'Training accuracy')
plt.plot(epochs, val_acc, 'b', label= 'Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

# epoch별 손실값 변화 그래프
plt.plot(epochs, acc, 'bo', label= 'Training loss')
plt.plot(epochs, val_acc, 'b', label= 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

> # **7. 남은 과제 및 challenge**

**>> Hyper-parameter 수정하여 최고 정확도 달성하기 <<**
*  수정가능한 Hyper-parameter: (소스코드의 **'##################수정가능'** 주석)


> # **Reference.**


1. 강아지 vs 고양이 데이터 (https://www.kaggle.com/c/dogs-vs-cats/data)
2. About ImageNet data: https://deepestdocs.readthedocs.io/en/latest/003_image_processing/0031/
3. About ILSVRC: https://bskyvision.com/425
4. "케라스 창시자에게 배우는 딥러닝", 프랑소와 숄레, 길벗, 2018)

